# Memory

- Memory is an integral part for an Agent - it enables an Agent to learn from previous experiences

- Here, we implement our own memory class in order to store memory in a suitable abstraction space from any modality - text / image / video / audio into memory chunks or graphs, and extract out the relevant chunks or graph components based on the task

- All Memory classes must implement MemoryTemplate, which contains the following core functions:
    - `append`: Takes in a list of memories, processes them and adds it to memory chunk / graph
    - `remove`: Takes in an existing memory and removes it
    - `reset`: Removes all memories
    - `retrieve`: Returns the top memories according to task
    
- Memory class can be used separately as part of a retrieval and storage pipeline that gets interfaced with the Agent via `Global Context` or directly into the task.

- Memory can also be passed into the `Agent` class as part of the `memory_bank`. If done this way, Agents will automatically use the `retrieve` function each time a subtask is done to append the relevant memories into the agent prompt. Note all other functions need to be done separately outside the `Agent` class, so it is best done within Agent wrappers

```python
class MemoryTemplate(ABC):
    """A generic template provided for all memories"""

    @abstractmethod
    def append(self, memory_list, mapper=None):
        """Appends multiple new memories"""
        pass

    @abstractmethod
    def remove(self, existing_memory):
        """Removes an existing_memory. existing_memory can be str, or triplet if it is a Knowledge Graph"""
        pass

    @abstractmethod
    def reset(self):
        """Clears all memories"""

    @abstractmethod
    def retrieve(self, task: str):
        """Retrieves some memories according to task"""
        pass
```



## Install new dependencies

In [1]:
%pip install sqlite-vec

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Some Initial Imports

In [2]:
import agentjo as aj


## Define your Memory Class Here
- Implement your Memory that overwrites all the required abstracted functions above
- The default Memory Class will be for a Sync Agent

In [3]:
import sqlite3
import sqlite_vec

def get_embedding(text:str):
  from openai import OpenAI
  client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
  cleaned_text = text.replace("\n", " ")
  embedding = client.embeddings.create(input=[cleaned_text], model="llama3.1").data[0].embedding
  return embedding

class SqliteMemory(aj.MemoryTemplate):
    ''' Memory in a SQL database, retrieves using OpenAI embeddings '''
    def __init__(self, memory_list = None, top_k: int = 3, sqlite_database=":memory:", embedding_procedure=async_get_embedding):
        ''' Inputs:
        memory_list: Initial list of memories to store
        top_k: Number of memories to retrieve
        sqlite_database: path to the sqlite database
        embedding_procedure: the procedure to get embedding value for text.
        '''
        self.top_k = top_k
        self.embedding_procedure = embedding_procedure
        self.embedding_length = len(self.embedding_procedure("foo"))

        self.database = sqlite3.connect(sqlite_database)
        self.database.enable_load_extension(True)
        sqlite_vec.load(self.database)
        self.database.enable_load_extension(False)
        cur = self.database.cursor()
        cur.execute(f"CREATE VIRTUAL TABLE vec_memories USING vec0(memory_embedding float[{self.embedding_length}])")
        cur.execute("create table memories(memory text)")
        if memory_list is not None:
            self.append(memory_list)

    def append(self, memory_list, mapper=None):
        """Adds a list of memories"""
        if not isinstance(memory_list, list):
            memory_list = [memory_list]
        cur = self.database.cursor()
        for memory in memory_list:
          embedding = self.embedding_procedure(memory)
          cur.execute("INSERT INTO memories(memory) VALUES (?)", [memory])
          cur.execute('insert into vec_memories(rowid, memory_embedding) VALUES (?, ?)', [cur.lastrowid, sqlite_vec.serialize_float32(embedding)])

    def remove(self, memory_to_remove):
        """Removes a memory"""
        cur = self.database.cursor()
        result = cur.execute("select rowid from memories WHERE memory = ?;", [memory_to_remove])
        rowid = result.fetchone()[0]
        cur.execute("delete from memories WHERE rowid = ?;", [rowid])
        cur.execute("delete from vec_memories WHERE rowid = ?;", [rowid])

    def reset(self):
        """Clears all memory"""
        cur = self.database.cursor()
        cur.execute("DELETE FROM vec_memories;")
        cur.execute("DELETE FROM memories;")
        
    def retrieve(self, task: str) -> list:
        task_embedding = self.embedding_procedure(task)
        """Performs retrieval of top_k similar memories according to embedding similarity"""
        cur = self.database.cursor()
        rows = cur.execute(
            """
              with matches as (SELECT rowid, distance FROM vec_memories WHERE memory_embedding MATCH ? AND k = ? ORDER BY distance)
              select memory from matches left join memories on memories.rowid = matches.rowid;
            """,
            [sqlite_vec.serialize_float32(task_embedding), self.top_k]
        ).fetchall()
        return [row[0] for row in rows]



### Use Memory without the Agent

In [4]:
memory = SqliteMemory(['hello', 'goodbye', 'good night', 'good morning'], top_k = 1)

In [5]:
memory.retrieve('good day')

['good morning']

### Use Memory in an Agent's Memory Bank

In [6]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')

    response = client.chat.completions.create(
        model='llama3.1',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [7]:
def greet_user(greeting: str) -> str:
    ''' Outputs greeting to the user '''
    return greeting

In [8]:
agent = aj.Agent('Greeting Generator', 'Greets user with the greeting present in Greet_Memory', 
              llm = llm).assign_functions([greet_user])
agent.memory_bank['Greet_Memory'] = SqliteMemory(['hello', 'goodbye', 'good night', 'good morning'], top_k = 1)

In [9]:
agent.run('It is a bright sunny day')

Observation: It is a bright sunny day
Thoughts: Consider how to respond to the weather description.
Subtask identified: Output greeting based on Greet_Memory
Calling function greet_user with parameters {'greeting': 'good morning'}
> {'output_1': 'good morning'}

Observation: The task mentions it being a bright sunny day, which suggests a morning scenario.
Thoughts: Considering the morning context, we can use the greeting "good morning" to acknowledge the user.
Subtask identified: Output the greeting "good morning" to the user.
Calling function greet_user with parameters {'greeting': 'good morning'}
> {'output_1': 'good morning'}

Observation: It is a bright sunny day
Thoughts: Consider how to acknowledge the weather in response.
Subtask identified: Output greeting based on Greet_Memory for the current time of day.
Calling function greet_user with parameters {'greeting': 'good morning'}
> {'output_1': 'good morning'}

Observation: It has been observed that the user wants to acknowledge 

[{'output_1': 'good morning'},
 {'output_1': 'good morning'},
 {'output_1': 'good morning'},
 {'output_1': 'good morning'},
 {'output_1': 'good morning'}]

# Async Memory
- Async memory is possible. Async memory must have async for the functions involving async processes
- Start your Async memory class name with Async

- **NOTE: When you contribute memory classes, it is not necessary to contribute the Async version**
- **This is only for more production-based use cases**

In [10]:
import sqlite3
import sqlite_vec

async def async_get_embedding(text:str):
  from openai import OpenAI
  client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
  cleaned_text = text.replace("\n", " ")
  result = await client.embeddings.create(input=[cleaned_text], model="llama3.1").data[0].embedding
  return result.data[0].embedding

class AsyncSqliteMemory(aj.MemoryTemplate):
    ''' Memory in a SQL database, retrieves using OpenAI embeddings '''
    def __init__(self, memory_list = None, top_k: int = 3, sqlite_database=":memory:", embedding_procedure=async_get_embedding):
        ''' Inputs:
        memory_list: Initial list of memories to store
        top_k: Number of memories to retrieve
        sqlite_database: path to the sqlite database
        embedding_procedure: the procedure to get embedding value for text.
        '''
        self.top_k = top_k
        self.embedding_procedure = embedding_procedure
        self.embedding_length = len(self.embedding_procedure("foo"))

        self.database = sqlite3.connect(sqlite_database)
        self.database.enable_load_extension(True)
        sqlite_vec.load(self.database)
        self.database.enable_load_extension(False)
        cur = self.database.cursor()
        cur.execute(f"CREATE VIRTUAL TABLE vec_memories USING vec0(memory_embedding float[{self.embedding_length}])")
        cur.execute("create table memories(memory text)")
        if memory_list is not None:
            self.append(memory_list)

    def append(self, memory_list, mapper=None):
        """Adds a list of memories"""
        if not isinstance(memory_list, list):
            memory_list = [memory_list]
        cur = self.database.cursor()
        for memory in memory_list:
          embedding = self.embedding_procedure(memory)
          cur.execute("INSERT INTO memories(memory) VALUES (?)", [memory])
          cur.execute('insert into vec_memories(rowid, memory_embedding) VALUES (?, ?)', [cur.lastrowid, sqlite_vec.serialize_float32(embedding)])

    def remove(self, memory_to_remove):
        """Removes a memory"""
        cur = self.database.cursor()
        result = cur.execute("select rowid from memories WHERE memory = ?;", [memory_to_remove])
        rowid = result.fetchone()[0]
        cur.execute("delete from memories WHERE rowid = ?;", [rowid])
        cur.execute("delete from vec_memories WHERE rowid = ?;", [rowid])

    def reset(self):
        """Clears all memory"""
        cur = self.database.cursor()
        cur.execute("DELETE FROM vec_memories;")
        cur.execute("DELETE FROM memories;")
        
    def retrieve(self, task: str) -> list:
        task_embedding = self.embedding_procedure(task)
        """Performs retrieval of top_k similar memories according to embedding similarity"""
        cur = self.database.cursor()
        rows = cur.execute(
            """
              with matches as (SELECT rowid, distance FROM vec_memories WHERE memory_embedding MATCH ? AND k = ? ORDER BY distance)
              select memory from matches left join memories on memories.rowid = matches.rowid;
            """,
            [sqlite_vec.serialize_float32(task_embedding), self.top_k]
        ).fetchall()
        return [row[0] for row in rows]

### Use Memory without the Agent

### Use Memory in an Agent's Memory Bank